安装软件包

In [2]:
%pip install -U numpy tqdm tensorboardx pyyaml torch pillow "jax[cuda12]" flax

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/50/a2/daca2bc563e1fd53c33fbff1e33e84004639f7ad9e1a9a54370480a7780d/flax-0.10.2-py3-none-any.whl (424 kB)
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/5c/24/28d0bb21600a78e46754947333ec9a297044af884d360092eb8561575fe9/optax-0.2.4-py3-none-any.whl (319 kB)
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/87/32/3779fa524a2272f408ab51d869fde9ff1c0ca731eedd01e40436bcf7ba2c/orbax_checkpoint-0.11.0-py3-none-any.whl (360 kB)
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/be/79/94decc1a93a4b53be8ef1fada387bbc1cf89475fe8da5a6dae6c74bf1f01/tensorstore-0.1.71-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.8 MB)
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/42/d7/1ec15b46af6af88f19b8e5ffea08fa375d433c998b8a7639e76935c14f1f/markdown_it_py-3.0.0-py3-non

获取NEU Seg数据集

In [3]:
!git clone https://github.com/DHW-Master/NEU_Seg.git data && rm -rf data/.git

正克隆到 'data'...
remote: Enumerating objects: 8947, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 8947 (delta 0), reused 0 (delta 0), pack-reused 8945 (from 1)
接收对象中: 100% (8947/8947), 35.08 MiB | 5.38 MiB/s, 完成.


对数据集进行处理

In [2]:
import os
import numpy as np
from PIL import Image

# 文件夹路径
train_image_dir = 'data/images/training'
train_mask_dir = 'data/annotations/training'
test_image_dir = 'data/images/test'
test_mask_dir = 'data/annotations/test'

# 获取文件列表
train_images = sorted(os.listdir(train_image_dir))
train_masks = sorted(os.listdir(train_mask_dir))
test_images = sorted(os.listdir(test_image_dir))
test_masks = sorted(os.listdir(test_mask_dir))

# 确保文件对应
assert len(train_images) == len(train_masks), "训练图片和掩码数量不一致"
assert len(test_images) == len(test_masks), "测试图片和掩码数量不一致"

# 完整路径
train_image_paths = [os.path.join(train_image_dir, img) for img in train_images]
train_mask_paths = [os.path.join(train_mask_dir, msk) for msk in train_masks]
test_image_paths = [os.path.join(test_image_dir, img) for img in test_images]
test_mask_paths = [os.path.join(test_mask_dir, msk) for msk in test_masks]

def load_images(paths):
    data = []
    for path in paths:
        img = Image.open(path).convert('RGB')
        img_array = np.array(img)
        # img_array = np.transpose(img_array, (2, 0, 1))  # 将通道轴移到最前面
        data.append(img_array)
    return np.array(data)

def load_masks(paths):
    data = []
    for path in paths:
        img = Image.open(path).convert('L')
        data.append(np.array(img))
    return np.array(data)

# 加载并保存训练集
train_images = load_images(train_image_paths)
train_masks = load_masks(train_mask_paths)
np.save('data/train_images.npy', train_images)
np.save('data/train_masks.npy', train_masks)

# 加载并保存测试集
test_images = load_images(test_image_paths)
test_masks = load_masks(test_mask_paths)
np.save('data/test_images.npy', test_images)
np.save('data/test_masks.npy', test_masks)

确认jax状态

In [3]:
import jax

# Show on which platform JAX is running.
print("JAX running on", jax.devices()[0].platform.upper())

JAX running on GPU


# 运行

In [44]:
!python main.py

Epoch:[1/500]
  0%|                                                   | 0/227 [00:00<?, ?it/s]2025-01-17 22:33:34.634812: W external/xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 25.21GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-01-17 22:33:35.270586: W external/xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 50.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-01-17 22:33:36.102375: W external/xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 19.44GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more

清理log

In [43]:
!rm -rf work_dir/*